# Data Preprocessing and Analysis

Dataset to process - shiksha_records.csv

Implement Logging

In [167]:
import logging
import sys
logging.basicConfig(level=logging.INFO,format=" %(asctime)s - [%(levelname)s]-%(message)s" \
                    ,handlers=[logging.FileHandler("shiksha_records_preprocessing.log"),
                               logging.StreamHandler(sys.stdout)])

logger = logging.getLogger(__name__)

# 1. Data Extraction

In [168]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas

create Spark object

In [169]:
spark = SparkSession.builder.appName("Shiksha_Records_Preprocess").config("spark.driver.bindAddress","10.0.2.15").getOrCreate()
logger.info("Spark object Created")

 2023-08-28 12:23:54,156 - [INFO]-Spark object Created


Read csv file from Hadoop hdfs


In [170]:
try:
    df = spark.read.csv("hdfs://localhost:9000/web_scraped_data/shiksha_records.csv",inferSchema=True,header=True)
    df.show(10)
except Exception as e:
    logger.error("csv from hadoop succssfully loaded")

+-----+--------------------+-----------+------------+-----------------+-------+--------------------+---------------+----------------+
|Sr No|        College Name|   Location|College Type|Number of courses|Ratings|      Exams Accepted|Total Fee Range| Average Package|
+-----+--------------------+-----------+------------+-----------------+-------+--------------------+---------------+----------------+
|    1|IIT Bombay - Indi...|     Mumbai|        Govt|      104 Courses|    4.6|GATEJEE MainJEE A...| ₹21.3 K - 10 L|₹8.4 - 55.37 LPA|
|    2|IIT Delhi - India...|      Delhi|        Govt|      117 Courses|    4.5|GATEJEE MainJEE A...|  ₹30 K - 8.7 L|  ₹12.5 - 22 LPA|
|    3|DTU - Delhi Techn...|      Delhi|        Govt|       78 Courses|    4.3|GATEJEE MainDASA ...|  ₹36 K - 6.4 L|     ₹9 - 13 LPA|
|    4|Manipal Institute...|    Manipal|         Pvt|       77 Courses|    4.3|METKarnataka PGCE...|₹2.2 L - 13.5 L| ₹6.2 - 8.88 LPA|
|    5|Amrita School of ...| Coimbatore|         Pvt|       32

# 2. Data Exploration

In [171]:
# getting rows and columns count
df_rows_count = df.count()
df_columns_count = len(df.columns)
print(f"Total Rows count: {df_rows_count}")
print(f"Total Columns count: {df_columns_count}")

Total Rows count: 10800
Total Columns count: 9


In [172]:
#Display dataframe structure
df.printSchema()

root
 |-- Sr No: integer (nullable = true)
 |-- College Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- College Type: string (nullable = true)
 |-- Number of courses: string (nullable = true)
 |-- Ratings: string (nullable = true)
 |-- Exams Accepted: string (nullable = true)
 |-- Total Fee Range: string (nullable = true)
 |-- Average Package: string (nullable = true)



In [173]:
df.describe().show()

+-------+-----------------+--------------------+--------+------------+-----------------+------------------+--------------+---------------+------------------+
|summary|            Sr No|        College Name|Location|College Type|Number of courses|           Ratings|Exams Accepted|Total Fee Range|   Average Package|
+-------+-----------------+--------------------+--------+------------+-----------------+------------------+--------------+---------------+------------------+
|  count|            10800|               10800|   10800|        8306|            10800|             10800|         10800|          10800|              6494|
|   mean|           5400.5|                null|    null|        null|             null|3.6628084280565463|          null|           null|3.4426192278576777|
| stddev|3117.835787850284|                null|    null|        null|             null|0.8938823071238124|          null|           null|0.9433969911930686|
|    min|                1|9.9 School of Com...|  Ad

Check Null Values

In [174]:
# Missing values count in all columns
missing_counts = df.select(*[(sum(col(c).isNull().cast("int"))).alias(c) for c in df.columns])
missing_counts.show()

+-----+------------+--------+------------+-----------------+-------+--------------+---------------+---------------+
|Sr No|College Name|Location|College Type|Number of courses|Ratings|Exams Accepted|Total Fee Range|Average Package|
+-----+------------+--------+------------+-----------------+-------+--------------+---------------+---------------+
|    0|           0|       0|        2494|                0|      0|             0|              0|           4306|
+-----+------------+--------+------------+-----------------+-------+--------------+---------------+---------------+



Removing dublicate values if any

In [175]:
# df = df.dropDuplicates(subset=["College Name"])

In [176]:
df.show()

+-----+--------------------+---------------+------------+-----------------+-------+--------------------+----------------+------------------+
|Sr No|        College Name|       Location|College Type|Number of courses|Ratings|      Exams Accepted| Total Fee Range|   Average Package|
+-----+--------------------+---------------+------------+-----------------+-------+--------------------+----------------+------------------+
|    1|IIT Bombay - Indi...|         Mumbai|        Govt|      104 Courses|    4.6|GATEJEE MainJEE A...|  ₹21.3 K - 10 L|  ₹8.4 - 55.37 LPA|
|    2|IIT Delhi - India...|          Delhi|        Govt|      117 Courses|    4.5|GATEJEE MainJEE A...|   ₹30 K - 8.7 L|    ₹12.5 - 22 LPA|
|    3|DTU - Delhi Techn...|          Delhi|        Govt|       78 Courses|    4.3|GATEJEE MainDASA ...|   ₹36 K - 6.4 L|       ₹9 - 13 LPA|
|    4|Manipal Institute...|        Manipal|         Pvt|       77 Courses|    4.3|METKarnataka PGCE...| ₹2.2 L - 13.5 L|   ₹6.2 - 8.88 LPA|
|    5|Amrita

In [177]:
df.printSchema()

root
 |-- Sr No: integer (nullable = true)
 |-- College Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- College Type: string (nullable = true)
 |-- Number of courses: string (nullable = true)
 |-- Ratings: string (nullable = true)
 |-- Exams Accepted: string (nullable = true)
 |-- Total Fee Range: string (nullable = true)
 |-- Average Package: string (nullable = true)



# 4.Data Transfromations

In [178]:
df = df.withColumnRenamed("Sr No", "sr_no") \
       .withColumnRenamed("College Name", "college_name") \
       .withColumnRenamed("Location", "location") \
       .withColumnRenamed("College Type", "college_type") \
       .withColumnRenamed("Number of courses", "number_of_courses") \
       .withColumnRenamed("Ratings", "ratings") \
       .withColumnRenamed("Exams Accepted", "exams_accepted") \
       .withColumnRenamed("Total Fee Range", "total_fee_range") \
       .withColumnRenamed("Average Package", "average_package")


In [179]:
df.show()

+-----+--------------------+---------------+------------+-----------------+-------+--------------------+----------------+------------------+
|sr_no|        college_name|       location|college_type|number_of_courses|ratings|      exams_accepted| total_fee_range|   average_package|
+-----+--------------------+---------------+------------+-----------------+-------+--------------------+----------------+------------------+
|    1|IIT Bombay - Indi...|         Mumbai|        Govt|      104 Courses|    4.6|GATEJEE MainJEE A...|  ₹21.3 K - 10 L|  ₹8.4 - 55.37 LPA|
|    2|IIT Delhi - India...|          Delhi|        Govt|      117 Courses|    4.5|GATEJEE MainJEE A...|   ₹30 K - 8.7 L|    ₹12.5 - 22 LPA|
|    3|DTU - Delhi Techn...|          Delhi|        Govt|       78 Courses|    4.3|GATEJEE MainDASA ...|   ₹36 K - 6.4 L|       ₹9 - 13 LPA|
|    4|Manipal Institute...|        Manipal|         Pvt|       77 Courses|    4.3|METKarnataka PGCE...| ₹2.2 L - 13.5 L|   ₹6.2 - 8.88 LPA|
|    5|Amrita

In [180]:
df = df.withColumn("number_of_courses",split(col("number_of_courses"), " ").getItem(0))
df = df.withColumn("number_of_courses",col("number_of_courses").cast("int"))
df = df.withColumn("ratings",col("ratings").cast("float"))
df.show()

+-----+--------------------+---------------+------------+-----------------+-------+--------------------+----------------+------------------+
|sr_no|        college_name|       location|college_type|number_of_courses|ratings|      exams_accepted| total_fee_range|   average_package|
+-----+--------------------+---------------+------------+-----------------+-------+--------------------+----------------+------------------+
|    1|IIT Bombay - Indi...|         Mumbai|        Govt|              104|    4.6|GATEJEE MainJEE A...|  ₹21.3 K - 10 L|  ₹8.4 - 55.37 LPA|
|    2|IIT Delhi - India...|          Delhi|        Govt|              117|    4.5|GATEJEE MainJEE A...|   ₹30 K - 8.7 L|    ₹12.5 - 22 LPA|
|    3|DTU - Delhi Techn...|          Delhi|        Govt|               78|    4.3|GATEJEE MainDASA ...|   ₹36 K - 6.4 L|       ₹9 - 13 LPA|
|    4|Manipal Institute...|        Manipal|         Pvt|               77|    4.3|METKarnataka PGCE...| ₹2.2 L - 13.5 L|   ₹6.2 - 8.88 LPA|
|    5|Amrita

In [181]:
df.printSchema()

root
 |-- sr_no: integer (nullable = true)
 |-- college_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- college_type: string (nullable = true)
 |-- number_of_courses: integer (nullable = true)
 |-- ratings: float (nullable = true)
 |-- exams_accepted: string (nullable = true)
 |-- total_fee_range: string (nullable = true)
 |-- average_package: string (nullable = true)



# 5. Data Analysis

In [182]:
import matplotlib.pyplot as plt

In [183]:
plot_df = df.toPandas()
plot_df.set_index(plot_df.columns[0], inplace=True)

In [184]:
plot_df

,college_name,location,college_type,number_of_courses,ratings,exams_accepted,total_fee_range,average_package
sr_no,,,,,,,,
1,IIT Bombay - Indian Institute of Technology,Mumbai,Govt,104,4.6,GATEJEE MainJEE AdvancedCOAPIIT JAMCEEDUGC NET...,₹21.3 K - 10 L,₹8.4 - 55.37 LPA
2,IIT Delhi - Indian Institute of Technology,Delhi,Govt,117,4.5,GATEJEE MainJEE AdvancedIIT JAMCEEDUCEEDUGC NE...,₹30 K - 8.7 L,₹12.5 - 22 LPA
3,DTU - Delhi Technological University,Delhi,Govt,78,4.3,GATEJEE MainDASA UGJAC DelhiCUET-PGCBSE 12thIS...,₹36 K - 6.4 L,₹9 - 13 LPA
4,Manipal Institute of Technology,Manipal,Pvt,77,4.3,METKarnataka PGCETGATEKCETDASA UGKMATCBSE 12th...,₹2.2 L - 13.5 L,₹6.2 - 8.88 LPA
5,"Amrita School of Engineering, Amrita Vishwa Vi...",Coimbatore,Pvt,32,4.1,JEE MainTNEAAEEEGATE +2,₹1.8 L - 18 L,₹4.5 - 6.41 LPA
...,...,...,...,...,...,...,...,...
10796,"School of Nursing, Command Hospital",Chandigarh,None,2,NaN,CBSE 12thPSEB 12th,– / –,None
10797,Sahajpath College of Pharmacy,West Bengal - Other,None,3,2.0,WBJEEJEE MainWBJEE JELET +2,₹3 L - 6.2 L,None
10798,"Maya Academy of Advanced Cinematics, Kolhapur",Kolhapur,Pvt,19,NaN,– / –,– / –,None


In [185]:
ratings_vs_type_df = plot_df.dropna(subset=["ratings"])

In [186]:
ratings_vs_type_df

,college_name,location,college_type,number_of_courses,ratings,exams_accepted,total_fee_range,average_package
sr_no,,,,,,,,
1,IIT Bombay - Indian Institute of Technology,Mumbai,Govt,104,4.6,GATEJEE MainJEE AdvancedCOAPIIT JAMCEEDUGC NET...,₹21.3 K - 10 L,₹8.4 - 55.37 LPA
2,IIT Delhi - Indian Institute of Technology,Delhi,Govt,117,4.5,GATEJEE MainJEE AdvancedIIT JAMCEEDUCEEDUGC NE...,₹30 K - 8.7 L,₹12.5 - 22 LPA
3,DTU - Delhi Technological University,Delhi,Govt,78,4.3,GATEJEE MainDASA UGJAC DelhiCUET-PGCBSE 12thIS...,₹36 K - 6.4 L,₹9 - 13 LPA
4,Manipal Institute of Technology,Manipal,Pvt,77,4.3,METKarnataka PGCETGATEKCETDASA UGKMATCBSE 12th...,₹2.2 L - 13.5 L,₹6.2 - 8.88 LPA
5,"Amrita School of Engineering, Amrita Vishwa Vi...",Coimbatore,Pvt,32,4.1,JEE MainTNEAAEEEGATE +2,₹1.8 L - 18 L,₹4.5 - 6.41 LPA
...,...,...,...,...,...,...,...,...
10786,Rajputana College of Pharmacy,Bangalore,Pvt,2,2.0,KCETCBSE 12thKarnataka 2nd PUC +2,₹1.1 L,None
10789,"Institute of Fire Service Engineering, MGM Uni...",Aurangabad,None,5,1.0,CBSE 12thISCMaharashtra HSC +1,₹40 K - 80 K,None
10791,Pavana College of Nursing,Bangalore,Pvt,2,2.0,KCETCBSE 12thKarnataka 2nd PUC +2,– / –,None


In [187]:
college_type_counts = ratings_vs_type_df.groupby("college_type").size()

# Display the count of each college type
print(college_type_counts)


college_type
Govt     915
PPP      151
Pvt     4746
dtype: int64


In [218]:
import plotly.express as px

# Filter out rows with None values in the 'ratings' column
ratings_vs_type_df_filtered = ratings_vs_type_df.dropna(subset=["ratings"])

new_ratings = ratings_vs_type_df_filtered.groupby('college_type')["ratings"].mean().reset_index()

# Creating an interactive bar plot using Plotly
fig = px.bar(new_ratings, x='college_type', y='ratings',
             labels={"college_type": "College Type", "ratings": "Ratings"},
             title="Ratings by College Type",color="college_type",color_discrete_sequence=["brown","green","blue"])

fig.update_layout(width=600,height=400)
# Show the plot
fig.show()



In [206]:

new_ratings_df = ratings_vs_type_df_filtered.groupby('college_type')["number_of_courses"].mean().reset_index()
# Create a bar plot using Matplotlib
# Creating an interactive bar plot using Plotly
fig = px.bar(new_ratings_df, x='college_type', y='number_of_courses',
             labels={"college_type": "College Type", "ratings": "Ratings"},
             title="Number of Courses by College Type")

fig.update_layout(width=600,height=400,barmode='group', bargap=0.5)
# Show the plot
fig.show()


In [189]:
spark.stop()